In [1]:
%matplotlib ipympl
def figure(name, nrows=1, ncols=1, *args, **kwargs):
    plt.close(name)
    return plt.subplots(nrows, ncols, num=name, *args, **kwargs)

import numpy as np
import pylab as plt
plt.style.use('default')

For more information have a look here: [https://pundit.pratt.duke.edu/wiki/Python:Ordinary_Differential_Equations](https://pundit.pratt.duke.edu/wiki/Python:Ordinary_Differential_Equations).

In [2]:
from ipywidgets import HBox, IntSlider, FloatSlider
from scipy.integrate import solve_ivp

We start from this equation
$$ my'' = -ky - Dy' \Leftrightarrow y'' + Dy' + ky = 0$$
for a damped harmonic oscillator and transform it to a system of first order:
$$ 
    y' = y_0' = y_1 \\
    y'' = y_1' = -\frac{k}{m}y_0 -\frac{D}{m}y_1
$$

In [3]:
# Implementation of the derivative we just derived
def derivative(t, y):
    return [y[1], 
            -k/m*y[0] - D/m*y[1]
           ]

We also know the analytical solution and can test our numerical results [https://de.wikipedia.org/wiki/Schwingung#Linear_ged%C3%A4mpfte_Schwingung](https://de.wikipedia.org/wiki/Schwingung#Linear_ged%C3%A4mpfte_Schwingung):

In [4]:
def analytical_solution(t):
    w0 = np.sqrt(k/m)
    d = D/(2*m)
    if d < w0:
        wd = np.sqrt(w0**2 - d**2)
        return np.exp(-d*t) * ((Y0[1] + d * Y0[0])/wd * np.sin(wd*t) + Y0[0]*np.cos(wd*t))
    elif d==w0:
        return (Y0[0] + (Y0[1] + Y0[1]*d)*t)*np.exp(-d*t)
    else:
        wd = np.sqrt(d**2 - w0**2)
        lp = -d + wd
        lm = -d - wd
        return (Y0[1] - lm*Y0[0])/ (2 * wd) * np.exp(lp*t) - (Y0[1] - lp*Y0[0])/ (2 * wd) * np.exp(lm*t)

In [5]:
times = np.linspace(0, 70, 1000)
m = 1
D = 0.15
k = 1
Y0 = [1, 1]

## First example

In [6]:
# You always give solve_ivp the derivative, the [starting_time, end_time], 
# the initial conditions (Y0) and a list of times at which you want to get the solution
sol = solve_ivp(derivative, [times[0], times[-1]], Y0, t_eval=times)
fig, ax = figure('example')
# y_0 is saved in sol.y[0] and y_1 in sol.y[1]. For an equation of order N, sol.y has N entries
ax.plot(sol.t, sol.y[0], label='x')
ax.plot(sol.t, analytical_solution(times), 'k--', label='x-analytic')
ax.plot(sol.t, sol.y[1], label='v')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Widget

In [7]:
def get_slider():
    slider_k = FloatSlider(
        orientation='horizontal', description='k: ',
        value=k, min=0, max=2, step = 0.05
    )
    slider_D = FloatSlider(
        orientation='horizontal', description='D: ',
        value=D, min=0, max=2, step = 0.05
    )

    sol = solve_ivp(derivative, [times[0], times[-1]], Y0, t_eval=times)
    fig, ax = figure(1, 1, 2, figsize=(10, 5))
    lines = [ax[0].plot(times, sol.y[0])[0],
             ax[0].plot(times, analytical_solution(sol.t), 'k--')[0],
            ax[0].plot(times, sol.y[1])[0],
            ax[1].plot(sol.y[0], sol.y[1])[0]]   
        
    ax[1].set_xlim([-1.5, 1.5])
    ax[1].set_ylim([-1.5, 1.5])
        
    def redraw():
        sol = solve_ivp(derivative, [times[0], times[-1]], Y0, t_eval=times)
        lines[0].set_data(times, sol.y[0])
        lines[1].set_data(times, analytical_solution(sol.t))
        lines[2].set_data(times, sol.y[1])
        lines[3].set_data(sol.y[0], sol.y[1])
        fig.canvas.draw()
        fig.canvas.flush_events()

    def update_k(change):
        global k
        k = change.new
        redraw()
        
    def update_D(change):
        global D
        D = change.new
        redraw()
        
    slider_k.observe(update_k, names='value')
    slider_D.observe(update_D, names='value')
    return HBox([slider_k, slider_D])

get_slider()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Driven Oscillator

Now, we add a driving term
$$ my'' = -ky - Dy' + F_0\cos(\omega t) $$

In [8]:
def derivative2(t, y):
    return [y[1], 
            -k/m*y[0] - D/m*y[1] + np.cos(0.5 * t)
           ]

In [9]:
times = np.linspace(0, 200, 1000)
m = 1
D = 0.05
k = 2
Y0 = [1, 1]

In [10]:
# You always give solve_ivp the derivative, the [starting_time, end_time], 
# the initial conditions (Y0) and a list of times at which you want to get the solution
sol = solve_ivp(derivative2, [times[0], times[-1]], Y0, t_eval=times)
fig, ax = figure('example')
# y_0 is saved in sol.y[0] and y_1 in sol.y[1]. For an equation of order N, sol.y has N entries
ax.plot(sol.t, sol.y[0], label='x')
ax.plot(sol.t, sol.y[1], label='v')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …